# Analyze dynophores

## Aim of this notebook

Explore how to analyze dynophores in a Jupyter notebook!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
import json
from pathlib import Path

import numpy as np
import pandas as pd

import dynophores

## Read data files

In [3]:
DATA = Path('../data/out/DynophoreApp')

In [4]:
dynophore_files = [file for file in DATA.glob('*')]
dynophore_files[:3]

[PosixPath('../data/out/DynophoreApp/1KE7-1_Data__superFeature_HBA[4619]%12.3.txt'),
 PosixPath('../data/out/DynophoreApp/1KE7-1_Data__superFeature_HBA[4618]%0.2.txt'),
 PosixPath('../data/out/DynophoreApp/1KE7-1_Data__superFeature_AR[4622,4615,4623,4613,4614,4621]%4.0.txt')]

### Json

In [7]:
# Initialize dynophore
dynophore = {'dynophore': []}

# Get superfeatures
superfeature_names = set([file.stem.split('_')[4].split('%')[0] for file in dynophore_files])

for superfeature_name in superfeature_names:
    
    # Get all files with this superfeature
    superfeature_files = [file for file in dynophore_files if superfeature_name in file.stem]
    
    # Get superfeature file
    assert len([file for file in superfeature_files if len(file.stem.split('_')) == 5]) == 1
    superfeature_file = [file for file in superfeature_files if len(file.stem.split('_')) == 5][0]
    
    # Set superfeature
    superfeature = {'name': superfeature_name, 'file': str(superfeature_file), 'env_partners': []}
    
    # Get environmental partner files for this superfeature
    for file in superfeature_files:
        if len(file.stem.split('_')) == 7:
            env_partner_file = file
            env_partner_name = env_partner_file.stem.split('_')[6].split('%')[0]
            env_partner = {'name': env_partner_name, 'file': str(env_partner_file)}
            superfeature['env_partners'].append(env_partner)
        
    dynophore['dynophore'].append(superfeature)

In [8]:
with open(DATA / '..' / 'dynophore_metadata.json', 'w') as f:
    json.dump(dynophore, f)

### Superfeatures

#### Superfeatures occurrence

In [ ]:
occurrence_superfeatures = pd.concat(
    [pd.read_csv(superfeature['file'], header=None) for superfeature in dynophore['dynophore']], 
    axis=1
)
occurrence_superfeatures.columns = [superfeature['name'] for superfeature in dynophore['dynophore']]
occurrence_superfeatures.head()

In [ ]:
type(occurrence_superfeatures.iloc[0, 0])

### Interactions

In [ ]:
def read_interactions(dynophore, type='occurrence'):
    
    types = ['occurrence', 'distance']
    if type == 'occurrence':
        type_ix = 1
    elif type == 'distance':
        type_ix = 0
    else:
        raise ValueError(f'Wrong type. Select from: {", ".join(types)}')
    
    interactions = defaultdict()

    for superfeature in dynophore['dynophore']:

        interaction = pd.concat(
            [pd.read_csv(env_partner['file'], header=None)[type_ix] for env_partner in superfeature['env_partners']], 
            axis=1
        )
        interaction.columns = [env_partner['name'] for env_partner in superfeature['env_partners']]
        interactions[superfeature['name']] = interaction

    return interactions

#### Interactions occurrence

In [ ]:
interactions_occurrence = read_interactions(dynophore, type='occurrence')
print(interactions_occurrence.keys())
interactions_occurrence['HBD[4612]'].head()

In [ ]:
type(interactions_occurrence['HBD[4612]'].iloc[0, 0])

#### Interactions distances

In [ ]:
interactions_distance = read_interactions(dynophore, type='distance')
print(interactions_distance.keys())
interactions_distance['HBD[4612]'].head()

In [ ]:
type(interactions_distance['HBD[4612]'].iloc[0, 0])

### `Dynophore` instance

In [ ]:
dynophore = dynophores.Dynophore()

In [ ]:
dynophore._get_file_components(dynophore_files[0])

In [ ]:

dynophore.from_file(DATA)
    
    

In [ ]:
dynophore._get_file_components()

In [ ]:
dynophore = from_files(DATA)

print(dynophore.id)
print(dynophore.superfeatures[2].id)
print(dynophore.superfeatures[2].feature_type)
print(dynophore.superfeatures[2].atom_numbers)
print(dynophore.superfeatures[2].occurrences)
print(dynophore.superfeatures[2].envpartners[0].id)
print(dynophore.superfeatures[2].envpartners[0].residue_name)
print(dynophore.superfeatures[2].envpartners[0].residue_number)
print(dynophore.superfeatures[2].envpartners[0].chain)
print(dynophore.superfeatures[2].envpartners[0].atom_numbers)
print(dynophore.superfeatures[2].envpartners[0].occurrences)
print(dynophore.superfeatures[2].envpartners[0].distances)